In [1]:
import equinox as eqx
import gym
import jax
import jax.numpy as jnp
import jax.random as jrandom
import numpy as np
import optax
import sys
import timeit
import wandb

from argparse import Namespace
from functools import partial
from jax import grad, jit, vmap
from typing import Sequence, Tuple, Optional

from jax_learning.buffers.ram_buffers import NextStateNumPyBuffer
from jax_learning.buffers.utils import batch_flatten, to_jnp
from jax_learning.constants import DISCRETE, CONTINUOUS
from jax_learning.rl_utils import interact, evaluate

from jax_learning.models import Policy, ActionValue, MLP, StochasticPolicy, Model
from jax_learning.models.policies import MLPSquashedGaussianPolicy

In [ ]:
wandb.init(project="test_jax_rl", group="reacher-sac_test")
wandb.define_metric("episodic_return", summary="max")

In [ ]:
cfg_dict = {
    # Environment setup
    "env": "Reacher-v2",
    "seed": 0,
    "render": False,
    
    # Experiment progress
    "load_step": 0,
    "log_interval": 5000,
    
    # Learning hyperparameters
    "max_timesteps": 1000000,
    "buffer_size": 1000000,
    "buffer_warmup": 1000,
    "num_gradient_steps": 1,
    "batch_size": 64,
    "max_grad_norm": 10.,
    "gamma": 0.99,
    "update_frequency": 4,
    
    # Actor
    "actor_lr": 3e-4,
    "actor_update_frequency": 1,
    
    # Critic
    "critic_lr": 3e-4,
    "target_update_frequency": 1,
    "tau": 0.005, # This is for polyak averaging of target network
    
    # Temperature
    "alpha_lr": 3e-4,
    "init_alpha": 1.0,
    "target_entropy": "auto",
    
    # Model architecture
    "hidden_dim": 256,
    "num_hidden": 2,
    
    # Evaluation
    "eval_cfg": {
        "max_episodes": 100,
        "seed": 1,
        "render": True,
    }
}
cfg = Namespace(**cfg_dict)
eval_cfg = Namespace(**cfg.eval_cfg)
wandb.config = cfg_dict

In [ ]:
np.random.seed(cfg.seed)

In [ ]:
env = gym.make(cfg.env)

In [ ]:
cfg.obs_dim = env.observation_space.shape
cfg.act_dim = env.action_space.shape
cfg.action_space = CONTINUOUS

In [ ]:
cfg.h_state_dim = (1,)
cfg.rew_dim = (1,)

In [ ]:
cfg

In [ ]:
cfg.buffer_rng = np.random.RandomState(cfg.seed)
cfg.env_rng = np.random.RandomState(cfg.seed)
cfg.agent_key, cfg.model_key = jrandom.split(jrandom.PRNGKey(cfg.seed), num=2)
eval_cfg.env_rng = np.random.RandomState(eval_cfg.seed)

In [2]:
from jax_learning.models.q_functions import MLPQ

In [3]:
obs_dim, act_dim, hidden_dim, num_hidden, keys = (1,), (1,), 2, 2, jrandom.split(jrandom.PRNGKey(0))

In [4]:
qs = [MLPSoftmaxQ(obs_dim, act_dim, hidden_dim, num_hidden, key) for key in keys]

TypeError: ActionValue.__init__() takes 1 positional argument but 3 were given

In [ ]:
class MultiQ(ActionValue):
    obs_dim: int
    act_dim: int
    qs: Sequence[ActionValue]

    def __init__(self,
                 obs_dim: Sequence[int],
                 act_dim: Sequence[int],
                 qs: Sequence[ActionValue]):
        self.obs_dim = int(np.product(obs_dim))
        self.act_dim = int(np.product(act_dim))
        self.qs = qs

    def q_values(self,
                 obs: np.ndarray,
                 h_state: np.ndarray,
                 act: Optional[np.ndarray]=None) -> Tuple[np.ndarray, np.ndarray]:
        q_vals, h_states = jax.vmap(Model.apply_function, in_axes=[0, None, None, None])(self.qs, obs, h_state, act)
        return q_vals, h_states


In [ ]:

model = MultiQ()

In [ ]:
params, static = eqx.partition(model, filter_spec=lambda x: isinstance(x, MLP))
print(type(params))
print(params)

In [ ]:
from jax_learning.learners import LearnerWithTargetNetwork
class SAC(LearnerWithTargetNetwork):
    def __init__(self,
                 model: eqx.Module,
                 target_model: eqx.Module,
                 opt: optax.GradientTransformation,
                 buffer: ReplayBuffer,
                 cfg: Namespace):
        super().__init__(model, target_model, opt, buffer, cfg)
        
        self._step = cfg.load_step
        self._batch_size = cfg.batch_size
        self._num_gradient_steps = cfg.num_gradient_steps
        self._gamma = cfg.gamma
        
        self._buffer_warmup = cfg.buffer_warmup
        self._update_frequency = cfg.update_frequency
        self._actor_update_frequency = cfg.actor_update_frequency
        self._target_update_frequency = cfg.target_update_frequency
        self._tau = cfg.tau
        self._omega = cfg.omega


In [ ]:
buffer = NextStateNumPyBuffer(
    buffer_size=cfg.buffer_size,
    obs_dim=cfg.obs_dim,
    h_state_dim=cfg.h_state_dim,
    act_dim=(1,) if cfg.action_space == DISCRETE else cfg.act_dim,
    rew_dim=cfg.rew_dim,
    rng=cfg.buffer_rng,
)

model = MLPSoftmaxQ(obs_dim=cfg.obs_dim,
                    act_dim=cfg.act_dim,
                    hidden_dim=cfg.hidden_dim,
                    num_hidden=cfg.num_hidden,
                    key=cfg.model_key)

target_model = MLPSoftmaxQ(obs_dim=cfg.obs_dim,
                           act_dim=cfg.act_dim,
                           hidden_dim=cfg.hidden_dim,
                           num_hidden=cfg.num_hidden,
                           key=cfg.model_key)

opt = optax.chain(
    optax.clip_by_global_norm(cfg.max_grad_norm),  # Clip by the gradient by the global norm
    optax.scale_by_adam(),  # Use the updates from adam
    optax.scale(-1.0) # Gradient descent
)

learner = QLearning(model=model,
                    target_model=target_model,
                    opt=opt,
                    buffer=buffer,
                    cfg=cfg)

agent = EpsilonGreedyAgent(model=model,
                           buffer=buffer,
                           learner=learner,
                           init_eps=cfg.init_eps,
                           min_eps=cfg.min_eps,
                           eps_decay=cfg.eps_decay,
                           eps_warmup=cfg.eps_warmup,
                           key=cfg.agent_key)

In [ ]:
%wandb

In [ ]:
interact(env, agent, cfg)

In [ ]:
evaluate(env, agent, eval_cfg)

In [ ]:
assert 0

In [ ]:
buffer

In [ ]:
buffer.observations

In [ ]:
np.roll(buffer.next_observations, 1, axis=0)

In [ ]:
np.concatenate((buffer.observations - np.roll(buffer.next_observations, 1, axis=0), buffer.dones), axis=1)

In [ ]:
buffer.sample_with_next_obs(3, buffer.next_observations[19], buffer.hidden_states[0])